In [11]:
#Functions for parsing columns of the spreadsheet on Santa Clara County Senior Meals
import re

def fcnparse_opendays(stringopendays):
    print(stringopendays)
    
def fcnparse_siteaddress(csvaddress):
    myoutput = nominatim.query(csvaddress+ ' California')
    if len(myoutput.toJSON())>0:
        myjson = myoutput.toJSON()[0]
    
        #print(myjson)
        #print(type(myjson))
        full_address = myjson['display_name']
        #print(full_address)
        #print(type(full_address))

        tmp = full_address.split(', ')
        #print(tmp)
       
        if tmp[0].isnumeric():
            address = tmp[0]+" "+tmp[1]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]            
        elif tmp[1].isnumeric():
            address = tmp[1]+" "+tmp[2]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]
        else:
            address = csvaddress+" ######CHECK ME######"
            city = "######CHECK ME######"
            zipcode = "######CHECK ME######"
            state = "California"
    else:
        full_address = ""
        address = csvaddress+"######CHECK ME######"
        city = "######CHECK ME######"
        zipcode = "######CHECK ME######"
        state = "California"
        
    return full_address, address, city, state, zipcode

def fcnparse_opendays(attributesOPENDAYS):
    return ["Monday","Tuesday","Thursday"]

def fcnparse_opentimes(attributesOPENTIMES):
    return "2:00 PM", "6:00 PM"

def fcnparse_mealtypes(attributesMEALTYPES):
    #print(attributesMEALTYPES)
    mealoptions = []
    if re.search('Take-Out', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Takeout", ]
    if re.search('Delivery', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Delivery", ]  
    if re.search('Curbside', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Curbside Pickup", ]   

    #print(mealoptions)
    return mealoptions


In [19]:
#Code to scrape Santa Clara County senior meal sites
import requests
import csv
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()

post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'
post_url_phones = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/phones'

post_headers = {
    'Authorization' : 'Bearer keyFaCMgt8zX71GnJ',
    'Content-Type': 'application/json'
}

csvfilename = open('/Users/Ashwin/Documents/BAC/20200919cfsj/Santa Clara Scrape 10_1 - Food Senior Meal Sites.csv')
csv_f = csv.reader(csvfilename)

#Skip header row
count = 1
for row in csv_f:
    if count == 1:
        break

#Loop over all rows and parse out the columns in each row
count = 1
for row in csv_f:
    #print(row)
    rowcount = 0
    for elem in row:
        print(rowcount, elem)
        rowcount = rowcount+1
    
    #Parse data for PHONES sheet: attributes.PHONENUMBER
    attributesPHONENUMBER = row[13]
    airtablePHONES = {"fields": {"number": attributesPHONENUMBER}}

    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES, 
    #attributes.SITESTATUS, attributes.DESCRIPTION, attributes.EMAIL, attributes.WEBLINK
    attributesSITENAME = row[4]
    attributesMEALTYPES = fcnparse_mealtypes(row[9])
    if row[9]=="Boxed Lunches":
        attributesFREEFOODAREA = ["Lunch"]
    else:
        attributesFREEFOODAREA = []
    attributesSITESTATUS = row[11]
    attributesDESCRIPTION = row[12]+" "+row[22]    
    attributesEMAIL = row[14]
    attributesWEBLINK = row[15]
    if attributesWEBLINK == "N/A":
        attributesWEBLINK = ""
    
    airtableSERVICES = {"fields": {"Name": attributesSITENAME, 
                                   "service_options": attributesMEALTYPES, 
                                   "freefood_area": attributesFREEFOODAREA, 
                                   "active": attributesSITESTATUS,
                                   "Description": attributesDESCRIPTION,
                                   "email": attributesEMAIL,
                                   "url": attributesWEBLINK}}

    
    #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
    attributesSITEADDRESS = row[5]
    full_address, address, city, state, zipcode = fcnparse_siteaddress(attributesSITEADDRESS)
    #print(full_address)
    #print(count+1, address, city, state, zipcode, '\n')
    airtableADDRESS = {"fields": {"address_1": address,"city": city,"State": state,"Zip Code": zipcode}} 
        
    #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
    attributesOPENDAYS = row[6]
    attributesOPENTIMES = row[7]
    opendays = fcnparse_opendays(attributesOPENDAYS)
    opensat, closesat = fcnparse_opentimes(attributesOPENTIMES)
    airtableSCHEDULE = {"fields": {"weekday": opendays, "opens_at": opensat, "closes_at": closesat}} 
    
    #Upload to airtable
    print((requests.post(post_url_phones, headers = post_headers, json = airtablePHONES)).status_code)    
    print((requests.post(post_url_services, headers = post_headers, json = airtableSERVICES)).status_code)    
    print((requests.post(post_url_address, headers = post_headers, json = airtableADDRESS)).status_code)    
    print((requests.post(post_url_schedule, headers = post_headers, json = airtableSCHEDULE)).status_code)

    count = count + 1
    #if count == 2:
    #    break    

0 0
1 116
2 116
3 116
4 Asian Americans for Community Involvement (AACI)
5 2400 Moorpark Ave Ste 300, San Jose, CA 95128
6 M - F
7 11:30 am to 12:00 pm
8 Year Round
9 Take-Out Available (Delivery available to existing participants. Call to apply for wait list)
10 Community Organization
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.
13 408-975-2374
14 Gerard.Manuel@aaci.org
15 N/A
16 Unincorporated
17 6
18 1588770037000
19 FME
20 Chinese
21 EOC-FSMS-0000000116
22 
23 -13574006.89
24 4483206.036
200
200
200
200
0 1
1 117
2 117
3 117
4 Billy De Frank LBGT Community  Center
5 938 The Alamada, San Jose, CA 95128
6 W&F
7 11:30
8 Year Round
9 Take-Out Available
10 Other Community Center
11 Closed
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribu

200
200
200
200
0 12
1 128
2 128
3 128
4 CSJ: Gardner Community Center
5 520 W. Virginia St, San Jose, CA 95110
6 M - F
7 11:30 am to 12:30 pm
8 Year Round
9 Take-Out Available
10 CSJ Community Center
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.
13 408-277-4761
14 rhonda.vargas@sanjoseca.gov
15 N/A
16 San Jose
17 3
18 1588770037000
19 FME
20 Mexican Cuisine
21 EOC-FSMS-0000000128
22 
23 -13569349.11
24 4483757.318
[nominatim] downloading data: 520 W. Virginia St, San Jose, CA 95110 California
200
200
200
200
0 13
1 129
2 129
3 129
4 CSJ: Mayfair Community Center
5 2039 Kammerer Ave, San Jose, CA 95113
6 M - F
7 11:45 am to 12:30 pm
8 Year Round
9 Take-Out Available
10 CSJ Community Center
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A

200
200
200
200
0 24
1 140
2 140
3 140
4 Yu Ai Kai
5 588 4th St, San Jose, CA 95112
6 M - F
7 12:00 pm to 12:30 pm
8 Year Round
9 Take-Out Available
10 Other Community Center
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.
13 408-297-4979
14 jmasuda@yuaikai.org
15 N/A
16 San Jose
17 3
18 1588770038000
19 FME
20 Japanese Cuisine
21 EOC-FSMS-0000000140
22 
23 -13569271.67
24 4487745.768
[nominatim] downloading data: 588 4th St, San Jose, CA 95112 California
200
200
200
200
0 25
1 141
2 141
3 141
4 Cubberly Community Center
5 4000 Middlefield Rd, Palo Alto, CA 94306
6 W
7 11:45
8 Year Round
9 Take-Out Available
10 Other Community Center
11 Closed
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help 

200
200
200
200
0 35
1 151
2 151
3 151
4 Villa Vasona
5 626 W. Parr Ave., Los Gatos, 95030
6 M, Th
7 11:30 am to 12:00 pm
8 Year Round
9 Take-Out Available
10 Community Organization
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.
13 408-354-0707
14 N/A
15 N/A
16 Los Gatos
17 0
18 1588770038000
19 FME
20 Cuisine
21 EOC-FSMS-0000000151
22 
23 -13577161.5
24 4475967.605
[nominatim] downloading data: 626 W. Parr Ave., Los Gatos, 95030 California
200
200
200
200
0 36
1 529
2 529
3 529
4 Masonic Center
5 461 Florence Street, Palo Alto 94301
6 M - F
7 11:45 am to 12:15 pm
8 Year Round
9 Take-Out Available
10 Community Organization
11 Open
12 The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the c

In [27]:
#Prototyping code for parsing addresses

from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
#myoutput = nominatim.query('1680 Foley ave')
#print(myoutput.toJSON())
csvaddress = '592 Dunholme Way, san jose'
myoutput = nominatim.query(csvaddress)
print(myoutput)
print(myoutput.toJSON())
print(len(myoutput.toJSON()))
myjson = myoutput.toJSON()[0]
print(myjson)
print(type(myjson))
full_address = myjson['display_name']
print(full_address)
print(type(full_address))

tmp = full_address.split(', ')
print(tmp)
firstelem = tmp[0]
print(firstelem)
print(firstelem.isnumeric())
if tmp[0].isnumeric():
    address = tmp[0]+" "+tmp[1]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]            
elif tmp[1].isnumeric():
    address = tmp[1]+" "+tmp[2]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]
else:
    address = csvaddress+" ######CHECK ME######"
    city = "######CHECK ME######"
    zipcode = "######CHECK ME######"
    state = "California"

print(address)
print(city)
print(state)
print(zipcode)

[nominatim] downloading data: 592 Dunholme Way, san jose
[]
0


IndexError: list index out of range

In [87]:
if ~len([]):
    print("yup empty")

yup empty


In [123]:
#blah = '1680'
#blah.isdigit()
print(1!=2)

True


In [11]:
#Prototyping code for parsing meal types

import re

if re.search('mandye', 'Mandy Pande', re.IGNORECASE):
    print("Yes found")
    
test_tup1 = [1, 3, 5] 
test_tup2 = [4,]
  
# printing original tuples 
print("The original tuple 1 : " + str(test_tup1)) 
print("The original tuple 2 : " + str(test_tup2)) 
  
# Ways to concatenate tuples 
# using + operator 
res = test_tup1 + test_tup2 
  
# printing result 
print("The tuple after concatenation is : " + str(res))

The original tuple 1 : [1, 3, 5]
The original tuple 2 : [4]
The tuple after concatenation is : [1, 3, 5, 4]


In [17]:
"test"=="test"

True